예제 1. 이미지 데이터 분류하기
==============================
> *Multi Layer Perceptron(MLP)* 을 구축하여 이미지 데이터(Fashion MNIST)를 분류해봅시다.


* 코드에 필요한 라이브러리를 호출합니다.
    1. Numpy
    2. Matplotlib
    3. torch (Pytorch)
    4. torchvision (이미지 처리 함수를 포함하는 pytorch 패키지)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

> 학습에 사용될 HyperParameter를 설정합니다. 

In [ ]:
batch_size = 100
learning_rate = 0.001
num_epochs = 5

> FashionMNIST 데이터셋을 다운로드합니다. torchvision의 datasets클래스에서 현재 많이 사용되는 공용 데이터셋을 배포하고있습니다.

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])


# 데이터셋 내려받기(fashion_mnist)
train_dataset  = torchvision.datasets.FashionMNIST("FashionMNIST/", download = True, train = True, transform = transform)
test_dataset  = torchvision.datasets.FashionMNIST("FashionMNIST/", download = True, train = False, transform = transform)

#fashion_mnist 데이터 데이터로더에 전달
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size)

#분류에 사용될 클래스 정의
labels_map = {0 : 'T-Shirt', 1 : 'Trouser', 2 : 'Pullover', 3 : 'Dress', 4 : 'Coat', 5 : 'Sandal', 6 : 'Shirt',
              7 : 'Sneaker', 8 : 'Bag', 9 : 'Ankle Boot'}

fig = plt.figure(figsize=(8,8));
columns = 4;
rows = 5;
for i in range(1, columns*rows +1):
    img_xy = np.random.randint(len(train_dataset));
    img = train_dataset[img_xy][0][0,:,:]
    fig.add_subplot(rows, columns, i)
    plt.title(labels_map[train_dataset[img_xy][1]])
    plt.axis('off')
    plt.imshow(img, cmap='gray')
plt.show()

> Neural Network Model을 정의합니다. 이번 예제의 모델은 MLP로 구축합니다.

In [ ]:
#심층신경망 모델 생성
class FashionDNN(nn.Module):
    def __init__(self):
        super(FashionDNN,self).__init__()
        self.fc1 = nn.Linear(in_features=784,out_features=256)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=256,out_features=128)
        self.fc3 = nn.Linear(in_features=128,out_features=10)

    def forward(self,input_data):
        out = input_data.view(-1, 784)
        out = F.relu(self.fc1(out))
        out = self.drop(out)
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

> Loss Function : CrossEntropy  
 Optimization Function : Adam

In [ ]:
model = FashionDNN()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

> 모델을 epoch 만큼 학습시킨 후 결과를 확인해봅시다.

In [ ]:
#모델 학습
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
    
        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)
        
        outputs = model(train)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1

        if not (count % 50):    
            total = 0
            correct = 0        
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)            
                test = Variable(images.view(100, 1, 28, 28))            
                outputs = model(test)            
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()            
                total += len(labels)
            
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        
        if not (count % 500):
            print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))

> 단일 이미지를 테스트해봅니다. 테스트 데이터셋에서 데이터 하나를 가져와 정확하게 예측하는지 확인합니다.

In [ ]:
x, y = next(iter(test_loader))
data_num = 0
img = x[data_num]
plt.imshow(img.view(28,28,1))
img = img.to(device)
groundtruth = labels_map[y[data_num].item()]
print('GroundTruth : ', groundtruth)
outputs = model(img)            
predictions = torch.max(outputs, 1)[1].to(device)
predict = labels_map[predictions.item()]
print('Prediction : ',predict)

> 같은 이미지에서 약간의 회전을 한 후 다시 테스트해보고 정확하게 예측하는지 확인합니다.

In [ ]:
t = transforms.RandomRotation(30)
new_img = t(img.detach().cpu())
plt.imshow(new_img.view(28,28,1))
new_img = new_img.to(device)
groundtruth = labels_map[y[data_num].item()]
print('GroundTruth : ', groundtruth)
outputs = model(new_img)            
predictions = torch.max(outputs, 1)[1].to(device)
predict = labels_map[predictions.item()]
print('Prediction : ',predict)